In [1]:
#Import of Python available libraries
import sculib

#mpi = sculib.scu()
#mpi.ip = '10.96.64.10'
sim = sculib.scu()
sim.ip = '10.96.254.85'

In [2]:
def mpi_t623(scu):
    print('Test 6.2.3')

    #Revisions:
    #0.1 initial write - could do with some kat sensor retrieval
    #0.2 rewrite for sculib

    #Common setup
    config_name = 'HN_TILT_TEST'
    
    V = [0.1,0.2,0.5,  1,1.5,  2,2.5, 3]
    P1 =  [1,  1,  1,  1,  3,  4,  7, 9]
    P2 =  [4,  6, 12, 22, 36, 48, 64, 78]
    P3 =  [5,  7, 13, 23, 39, 52, 71, 87]
    T1 = [31, 26, 23, 23, 25, 26, 28, 29]
    T2 = [40, 41, 42, 44, 46, 48, 50, 52]
    T3 = [71, 56, 47, 45, 49, 50, 54, 55]
    
    loop1 = 8
    loop2 = 5
    
    az_start = 0
    az_speed = 3

    elevations = [85, 50, 20]
    el_start = 85
    el_speed  = 0.1
    
    wait5 = 5
    wait10 = 10
    wait100 = 100
    sampling = 200
    scu.debug = False
    #debug test only remove these
    #'''
    loop1 = 5

    T1 = [1, 2, 3, 4, 5, 6, 7, 8]
    T2 = T1
    T3 = T1
    
    wait5 = 1
    wait10 = 1
    wait100 = 1
    sampling = 5000
    scu.debug = False
    #'''

    #B. Tilt Sensor tests [316-000000-043]
    '''
    1) Manually move the Dish to 85° Elevation, 0° Azimuth and de-activate all axes
    2) Manually measure and record the Pedestal wall temperature at the N/E/S/W positions 
    (consider using iButtons)
    3) Activate the Azimuth and Elevation axis
    • wait for 15 seconds to complete the activation process
    4) Start the ACU data recording on the SCU using logging configuration TBD,
    5) Command Azimuth to position P3(i) at V(i) °/sec, (refer to Table 1 for the different rates and positions)
    • Wait for T3(i) sec
    6) Repeat step 5) back to the starting position
    7) Command Azimuth to position P1(i) at V(i) °/sec,
    • Wait for T1(i) sec
    8) Command Azimuth to position P2(i) at V(i) °/sec,
    • Wait for T2(i) sec
    9) Repeat step 8) back to P1(i),
    10) Repeat steps 8) and 9) for a total of 5 cycles
    11) Command Azimuth to the starting position at V(i) °/sec,
    • Wait for T1(i) sec
    12) Repeat steps 5) to 11) for the different rates V(2) to V(8) as defined in Table 1
    13) Manually measure and record the Pedestal wall temperature at the N/E/S/W positions (consider using iButtons)
    14) Extract the data on the SCU @ 5Hz sampling
    15) Rename the file to TBD (eg. 6p2p3_i.csv)
    16) Repeat steps 1) to 15) for Elevation angle of 50°
    17) Repeat steps 1) to 15) for Elevation angle of 15°
    18) Download the MeerKAT weather station data for the test period
    '''
    for elevation in elevations:
        print("\n Elevation angle: {} \n".format(elevation))

        #Step 1
        #Manual move dish to  85° Elevation, 0° Azimuth
        print("Manual move to az 0 and el 85")
        scu.activate_dmc()
        scu.wait_duration(wait10*3)
        
        az_angle = az_start
        scu.abs_azimuth(az_angle, az_speed)
        scu.wait_duration(wait100)
        
        el_angle = elevation
        scu.abs_elevation(el_angle, el_speed)
        scu.wait_duration(wait100*8)
        
        scu.deactivate_dmc()
        scu.wait_duration(wait10*2) 
        #Step 2
        #Record pedestal wall temperatures
        print('Record pedestal wall temperatures')
        
        #Step 3
        scu.activate_dmc()
        scu.wait_duration(wait10*3)
        
        #Step 4 - start ACU recording on SCU
        if scu.logger_state() == 'RECORDING':
            print('WARNING, already recording - attempting to stop and start a fresh logger')
            scu.stop_logger()  
            scu.wait_duration(wait10)
        if scu.logger_state() == 'STOPPED':
            print('Starting logger with config: {}'.format(config_name))
            scu.start_logger(config_name)
            scu.wait_duration(wait10)

       # i=0
       # while i < loop1:
        for i, POS in enumerate(P1):
            print("\t Loop 1 cycle: {} P1 {}".format(i, POS))
        #    i+=1
        #    print("\t Loop 1 cycle: {} of {}".format(i, loop1))
            #Step 5 - Command Azimuth to position P3(i) at V(i) °/sec, Wait for T3(i) se
            az_angle = P3[i] 
            az_speed = V[i]
            scu.abs_azimuth(az_angle, az_speed)
            scu.wait_duration(T3[i])

            #Step  6) Repeat step 5) back to the starting position  ASSUME still at V[i], WAITING T3??
            az_angle = az_start 
            az_speed = V[i]
            scu.abs_azimuth(az_angle, az_speed)
            scu.wait_duration(T3[i])
            
            #Step 7) Command Azimuth to position P1(i) at V(i) °/sec,
            az_angle = P1[i] 
            az_speed = V[i]
            scu.abs_azimuth(az_angle, az_speed)
            scu.wait_duration(T1[i])
            
            j=0
            while j < loop2:

                j+=1
                print("\t \t Loop 2 cycle: {} of {}".format(j, loop2))
                #Step 8) Command Azimuth to position P2(i) at V(i) °/sec, Wait for T2(i) sec
                az_angle = P2[i] 
                az_speed = V[i]
                scu.abs_azimuth(az_angle, az_speed)
                scu.wait_duration(T2[i])

                #Step 9) Repeat step 8) back to P1(i), ASSUME still at V[i] wait T2[i]?
                az_angle = P1[i] 
                az_speed = V[i]
                scu.abs_azimuth(az_angle, az_speed)
                scu.wait_duration(T2[i])
            
                #Step 10) Repeat steps 8) and 9) for a total of 5 cycles
            #Step 11) Command Azimuth to the starting position at V(i) °/sec, Wait for T1(i) sec
            az_angle = az_start 
            az_speed = V[i]
            scu.abs_azimuth(az_angle, az_speed)
            scu.wait_duration(T1[i])           
            
        #Step 12) Repeat steps 5) to 11) for the different rates V(2) to V(8) as defined in Table 1
    #Step 13) Manually measure and record the Pedestal wall temperature at the N/E/S/W positions (consider using iButtons)
    #Step 14) Extract the data on the SCU @ 5Hz sampling
    scu.stop_logger()
    scu.wait_duration(wait10)
    print(scu.logger_state())
        
    #Step 15) Rename the file to TBD (eg. 6p2p3_i.csv)
    print("Rename the file to TBD (eg. 6p2p3_i.csv)")
    scu.save_session('6p2p3_i_' + str(elevation), interval_ms = sampling)
        
    #Step 16) Repeat steps 1) to 15) for Elevation angle of 50°
    #Step 17) Repeat steps 1) to 15) for Elevation angle of 15°
    #Step 18) Download the MeerKAT weather station data for the test period
    print("Download the MeerKAT weather station data for the test period")


In [3]:
#mpi_t623(mpi)
mpi_t623(sim)


Test 6.2.3

 Elevation angle: 85 

Manual move to az 0 and el 85
activate dmc
wait for 3.0s done *
abs az: 0.000 vel: 3.000
wait for 1.0s done *
abs el: 85.000 vel: 0.100
wait for 8.0s done *
deactivate dmc
wait for 2.0s done *
Record pedestal wall temperatures
activate dmc
wait for 3.0s done *
logger state 
logger state 
Starting logger with config: HN_TILT_TEST
start logger:  HN_TILT_TEST
wait for 1.0s done *
	 Loop 1 cycle: 0 P1 1
abs az: 5.000 vel: 0.100
wait for 1.0s done *
abs az: 0.000 vel: 0.100
wait for 1.0s done *
abs az: 1.000 vel: 0.100
wait for 1.0s done *
	 	 Loop 2 cycle: 1 of 5
abs az: 4.000 vel: 0.100
wait for 1.0s done *
abs az: 1.000 vel: 0.100
wait for 1.0s done *
	 	 Loop 2 cycle: 2 of 5
abs az: 4.000 vel: 0.100
wait for 1.0s done *
abs az: 1.000 vel: 0.100
wait for 1.0s done *
	 	 Loop 2 cycle: 3 of 5
abs az: 4.000 vel: 0.100
wait for 1.0s done *
abs az: 1.000 vel: 0.100
wait for 1.0s done *
	 	 Loop 2 cycle: 4 of 5
abs az: 4.000 vel: 0.100
wait for 1.0s done *
ab

wait for 2.0s done *
abs az: 1.000 vel: 0.200
wait for 2.0s done *
	 	 Loop 2 cycle: 1 of 5
abs az: 6.000 vel: 0.200
wait for 2.0s done *
abs az: 1.000 vel: 0.200
wait for 2.0s done *
	 	 Loop 2 cycle: 2 of 5
abs az: 6.000 vel: 0.200
wait for 2.0s done *
abs az: 1.000 vel: 0.200
wait for 2.0s done *
	 	 Loop 2 cycle: 3 of 5
abs az: 6.000 vel: 0.200
wait for 2.0s done *
abs az: 1.000 vel: 0.200
wait for 2.0s done *
	 	 Loop 2 cycle: 4 of 5
abs az: 6.000 vel: 0.200
wait for 2.0s done *
abs az: 1.000 vel: 0.200
wait for 2.0s done *
	 	 Loop 2 cycle: 5 of 5
abs az: 6.000 vel: 0.200
wait for 2.0s done *
abs az: 1.000 vel: 0.200
wait for 2.0s done *
abs az: 0.000 vel: 0.200
wait for 2.0s done *
	 Loop 1 cycle: 2 P1 1
abs az: 13.000 vel: 0.500
wait for 3.0s done *
abs az: 0.000 vel: 0.500
wait for 3.0s done *
abs az: 1.000 vel: 0.500
wait for 3.0s done *
	 	 Loop 2 cycle: 1 of 5
abs az: 12.000 vel: 0.500
wait for 3.0s done *
abs az: 1.000 vel: 0.500
wait for 3.0s done *
	 	 Loop 2 cycle: 2 of

wait for 3.0s done *
	 	 Loop 2 cycle: 5 of 5
abs az: 12.000 vel: 0.500
wait for 3.0s done *
abs az: 1.000 vel: 0.500
wait for 3.0s done *
abs az: 0.000 vel: 0.500
wait for 3.0s done *
	 Loop 1 cycle: 3 P1 1
abs az: 23.000 vel: 1.000
wait for 4.0s done *
abs az: 0.000 vel: 1.000
wait for 4.0s done *
abs az: 1.000 vel: 1.000
wait for 4.0s done *
	 	 Loop 2 cycle: 1 of 5
abs az: 22.000 vel: 1.000
wait for 4.0s done *
abs az: 1.000 vel: 1.000
wait for 4.0s done *
	 	 Loop 2 cycle: 2 of 5
abs az: 22.000 vel: 1.000
wait for 4.0s done *
abs az: 1.000 vel: 1.000
wait for 4.0s done *
	 	 Loop 2 cycle: 3 of 5
abs az: 22.000 vel: 1.000
wait for 4.0s done *
abs az: 1.000 vel: 1.000
wait for 4.0s done *
	 	 Loop 2 cycle: 4 of 5
abs az: 22.000 vel: 1.000
wait for 4.0s done *
abs az: 1.000 vel: 1.000
wait for 4.0s done *
	 	 Loop 2 cycle: 5 of 5
abs az: 22.000 vel: 1.000
wait for 4.0s done *
abs az: 1.000 vel: 1.000
wait for 4.0s done *
abs az: 0.000 vel: 1.000
wait for 4.0s done *
	 Loop 1 cycle: 4